In [ ]:
# ✅todo : 1. colab 용으로 requirements.txt을 하나 더 만든다.
# ✅ todo : 1. turnonselenium 내용을 colab용으로 추가한다.
# ✅todo : 1. blogger를 모듈화해서 사용하기 쉽게 만든다.
# ✅todo : 1. 쓰레딩 시키는 것도 함수화한다.
# ✅todo : 1. 코랩은 쓰레딩 몇개를 시키는게 적당한지를 확인한다 = 2개 (2코어라서)
# ✅todo : 1. 코랩크롤링 시에, user-agent 사용하도록 한다.
# todo : duckduckgo 관련자료 서칭 대체품 찾기
# todo : 쉬는 시간 가지도록 만들기
# todo : 만들어진 컨텐츠들 중에 document 없는 경우 찾기


# ✅todo : 1딸깍 테스트
# todo : 1딸깍 반복문에 쉬는 시간 부여하기
# todo : 2딸깍 테스트
# todo : 2딸깍 반복문에 쉬는 시간 부여하기
# todo : 3딸깍 테스트
# todo : 3딸깍 반복문에 쉬는 시간 부여하기

# ✅todo : 1딸깍 테스트 - colab
# todo : 2딸깍 테스트 - colab
# todo : 3딸깍 테스트 - colab

# todo : 추가 키워드 수집 작업 시 기존 csv를 덮어쓸지 말지를 결정하도록 해주세요. - 현재는 덮어씌우지 않는 것으로 되어있음.
# todo : 추가 키워드 수집 작업 시 기존 suitable keywords 리스트를 고려하도록 해주세요. (있으면 재반복해서 수행하지는 않는 것으로.)

# todo : 업로드가 성공적으로 수행되면은 옮기는 것으로 변경하세요.
# todo : 노출순위 따질 때 1) 한번정도는 아래로 스크롤해서 가져오기 2) 네이버 블로그도 순위에 포함시키기.
# todo : 글 생성에 사용한 키워드 저장하고 유사키워드 문지방수준  좀 줄여야 할 것 같아. 너무 관계없는 애들도 같이 묶이는 것 같어.
# todo : 블로그에 올릴 때, 성공한 글에 대해서만 uploaded로 옮기도록 해야할 것 같어.
# todo : suitable check를 할 때, 기존 screened_keywords에 있는 키워드들은 제외하도록 해야할 것 같아. 그러기 위해서는 키워드 수집 시에 collected_keywords에다가 suitability_checked 여부를 표시를 해두어야 할 것 같습니다.


# 기본실행

In [1]:
# 필요한 실행함수 정의
def thread_bloggers(func, bloggers, **kwargs):
    import threading

    threads = []
    for blogger in bloggers.values():
        # args 튜플에 blogger를 포함시키고, kwargs 딕셔너리를 threading.Thread에 전달
        thread = threading.Thread(target=func, args=(blogger,), kwargs=kwargs)
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
def create_partial_bloggers(total_bloggers:dict, selected_bloggers:list) :
  partial_bloggers = {}
  for blogger in selected_bloggers :
    partial_bloggers[blogger] = total_bloggers[blogger]
  return partial_bloggers


# 필요한 모듈 불러오기
from module.Crawler import Crawler
from module.File_manager import File_manager
from module.Blogger import Blogger
from module.AIAgent import AIAgent
from module.Uploader import Uploader

# 실행환경 설정
import warnings
warnings.filterwarnings("ignore")
verbose = True
isHeadless = True
isTest = input('test?, yes or no')


# blogger 클래스를 호출합니다.
if isTest == 'no' :
  blog_names = ['statifi', 'kindmom', 'sweetkiwi', 'takuz', 'morningbbobbo']
  bloggers = {}
  for blogname in blog_names:
      print(f'🌐 now loading : {blogname}.')
      bloggers[blogname] = Blogger(blogname = blogname, verbose = verbose, isHeadless = isHeadless)


# tester용 객체들을 만듭니다.
if isTest == 'yes':
  verbose = True
  isHeadless = True
  crawler_tester = Crawler(isHeadless = isHeadless, verbose = verbose)
  file_manager_tester = File_manager(blogname='tester', verbose=verbose)
  aiagent_tester = AIAgent(blogname='tester', verbose = verbose)
  uploader_tester = Uploader(blogname='tester', verbose = verbose)
  blogger_tester = Blogger(blogname='tester', verbose = verbose, isHeadless = isHeadless)

🌐 now loading : statifi.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
✅ 기존 벡터스토어 로드되었습니다.
✅ 기존 벡터스토어 로드되었습니다.
True
True
Data loaded from JSON
🌐 now loading : kindmom.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
✅ 기존 벡터스토어 로드되었습니다.
✅ 기존 벡터스토어 로드되었습니다.
True
True
[Errno 2] No such file or directory: 'c:\\hellodear\\tistory_ebook\\bloggers\\kindmom\\storage\\[[kindmom]]_((ko))_``collected_keywords``.json'
해당 블로그는 수집한 키워드 정보가 없습니다.
🌐 now loading : sweetkiwi.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
✅ 기존 벡터스토어 로드되었습니다.
✅ 기존 벡터스토어 로드되었습니다.
True
True
Data loaded from JSON
🌐 now loading : takuz.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
기존에 생성된 폴더가 존재합니다.
✅ 기존 벡터스토어 로드되었습니다.
✅ 기존 벡터스토어 로드되었습니다.
True
True
[Errno 2] No such file or directory: 'c:\\hellodear\\tistory_ebook\\bloggers\\takuz\\storage\\[[takuz]]_((ko))_``collected_keywords``.json'
해당 블로그는 수집한 키워드 정보가 없습니다.
🌐 now loading : 

In [2]:
for blogname in ['statifi', 'sweetkiwi'] :
    print(len(bloggqers[blogname].keyword_ai.vectorstore_list))

4400
999


# 클릭1 : 키워드 수집

쓰레딩


In [ ]:
# 환경설정

# 깊이 정의
depth = int(input('키워드를 수집할 깊이를 입력하세요. (최대 3 이상은 비추천) :'))

# 단어 정의
recommended = {
}

subjects_n_words_for_statifi = {
    "건강과 웰빙": ["요가", "마인드풀니스", "웰니스 리트릿", "건강한 간식", "명상", "필라테스", "건강한 식단", "건강 추세", "힐링", "건강 검진"],
}
subjects_n_words_for_kindmom = {
    "건강 및 웰니스 블로그": ["건강한 식단", "정신 건강", "스트레스 관리", "홈 피트니스", "명상 기법", "웰니스 라이프스타일", "영양 정보", "건강 검진 가이드", "체중 관리", "건강한 습관"],
}
subjects_n_words_for_sweetkiwi = {
    "재택 근무 및 자기 계발 블로그": ["재택 근무 팁", "자기계발 책 추천", "온라인 학습 플랫폼", "시간 관리 기술", "생산성 향상", "사이드 허슬", "재택 근무 환경", "프리랜서 가이드", "자기 관리", "모티베이션 증진"],
}
subjects_n_words_for_takuz = {
    "기업과 기업가": ["기업가 정신", "크라우드펀딩", "비즈니스 모델 혁신", "기술 벤처", "벤처 투자", "스타트업 생태계"],

}
subjects_n_words_for_morningbbobbo = {
    "기술 및 혁신 블로그": ["최신 기술 트렌드", "모바일 기기 리뷰", "인공 지능", "가상 현실", "사이버 보안", "프로그래밍 팁", "소프트웨어 개발", "기술 스타트업", "클라우드 컴퓨팅", "스마트 홈 기술", "블록체인 및 암호화폐", "자율주행 자동차", "5G 네트워크", "인터넷 오브 씽스", "데이터 사이언스 및 빅데이터", "에너지 기술 혁신", "로보틱스", "인간-컴퓨터 상호작용", "나노기술", "테크놀로지 인 인더스트리 4.0"]
}

# 각 블로거의 주제 및 키워드 데이터

total_subjects_n_words = {
    'statifi': subjects_n_words_for_statifi,
    'kindmom': subjects_n_words_for_kindmom,
    'sweetkiwi': subjects_n_words_for_sweetkiwi,
    'takuz': subjects_n_words_for_takuz,
    'morningbbobbo': subjects_n_words_for_morningbbobbo
}



# 함수정의
def collect_keywords_for_blogger(blogger, total_subjects_n_words, depth, save):
    subjects_n_words = total_subjects_n_words[blogger.blogname]
    blogger.collect_keywords(subjects_n_words=subjects_n_words, depth=depth, save=save, suitable_keywords_save = False)

if isTest == 'no':
    # for blogname in blog_names :
    #     collect_keywords_for_blogger(bloggers[blogname], total_subjects_n_words, depth = depth, save = True)


    # 2 bloggers
    partial_bloggers = create_partial_bloggers(bloggers, ['statifi','sweetkiwi'])

    # 실행
    thread_bloggers(
        func=collect_keywords_for_blogger,
        bloggers=partial_bloggers,
        total_subjects_n_words=total_subjects_n_words,
        depth=depth,
        save=True
    )

    # 2 bloggers
    partial_bloggers2 = create_partial_bloggers(bloggers, ['takuz', 'kindmom'])

    # 실행
    thread_bloggers(
        func=collect_keywords_for_blogger,
        bloggers=partial_bloggers2,
        total_subjects_n_words=total_subjects_n_words,
        depth=depth,
        save=True
    )

    partial_bloggers3 = create_partial_bloggers(bloggers, ['morningbbobbo'])

    # 실행
    thread_bloggers(
        func=collect_keywords_for_blogger,
        bloggers=partial_bloggers3,
        total_subjects_n_words=total_subjects_n_words,
        depth=depth,
        save=True
    )

if isTest == 'yes':
    subjects_n_words = {"tester" : {"파이어족" : ['파이어족'],
                                   "히키코모리" : ['히키코모리']}}
    collect_keywords_for_blogger(blogger_tester,  subjects_n_words, depth = 1, save = True)
    


tester 테스트

In [ ]:
# 단일 블로거로 테스트

test_blog = 'tester'
total_subjects_n_words = {test_blog : {'파이어족' : ['파이어족']} }
depth = 1

# 함수정의
def collect_keywords_for_blogger(blogger, total_subjects_n_words, depth, save):
    subjects_n_words = total_subjects_n_words[blogger.blogname]
    blogger.collect_keywords(subjects_n_words=subjects_n_words, depth=depth, save=save)


collect_keywords_for_blogger(blogger_tester, total_subjects_n_words, depth, save=True)

vectorstore_stored_keywords = blogger_tester.keyword_ai.vectorstore_list.index.to_list()
if len(vectorstore_stored_keywords) > 0 :
    print('🌐 키워드 수집이 완료되었습니다.')    
    blogger_tester.keyword_ai.vectorstore_clear()

# 클릭 2 : 포스팅 생성

순차진행

In [ ]:
# 환경설정
# num_contents_creation = int(input('생성할 컨텐츠 갯수를 입력하세요. 무한대생산은 엔터를 치세요 '))
num_contents_creation = 20

# 함수정의
def create_contents(blogger, num_contents_creation=5):
    print(f"Starting thread for {blogger.blogname}")
    blogger.create_contents(num_contents_creation)

for blogname in blog_names :
    create_contents(bloggers[blogname], num_contents_creation=num_contents_creation)

In [ ]:
for i in bloggers['statifi'].keyword_ai.vectorstore_list['data'].to_list() :
    a = bloggers['statifi'].keyword_ai.vectorstore.similarity_search(i)[0].page_content
    if a != i :
        print(a, i)


쓰레딩

In [ ]:
# 환경설정
num_contents_creation = int(input('생성할 컨텐츠 갯수를 입력하세요. 무한대생산은 엔터를 치세요 '))

# 함수정의
def create_contents(blogger, num_contents_creation=5):
    print(f"Starting thread for {blogger.blogname}")
    blogger.create_contents(num_contents_creation)

# 실행
thread_bloggers(create_contents, bloggers, num_contents_creation=num_contents_creation)

부분쓰레딩 (2 + 3)

In [20]:
# 환경설정
# num_contents_creation = int(input('생성할 컨텐츠 갯수를 입력하세요. 무한대생산은 엔터를 치세요 '))
num_contents_creation = 9999 

# 함수정의
def create_contents(blogger, num_contents_creation=5):
    print(f"Starting thread for {blogger.blogname}")
    blogger.create_contents(num_contents_creation)

partial_bloggers = create_partial_bloggers(bloggers, ['statifi', 'sweetkiwi'])
thread_bloggers(create_contents, partial_bloggers, num_contents_creation=num_contents_creation)
# partial_bloggers2 = create_partial_bloggers(bloggers, ['takuz', 'morningbbobbo', 'kindmom'])
# thread_bloggers(create_contents, partial_bloggers2, num_contents_creation=num_contents_creation)

Starting thread for statifi
🌐 글을 작성할 키워드를 가지고 오는 중... 
Starting thread for sweetkiwi
🌐 글을 작성할 키워드를 가지고 오는 중... 
여기1
여기1
여기2
이거를 삭제하라 햇슴 : ['자기주장']
여기3
여기4
여기2
이거를 삭제하라 햇슴 : ['embt']
여기3
여기4
🌐 글 관련 자료 수집 중 ... 
🌐 글 관련 자료 수집 중 ... 
✅ 벡토어스토어 내 새 문서 추가 완료
🌐 글 작성 중 ... 
여기1
여기2
✅ 벡토어스토어 내 새 문서 추가 완료
🌐 글 작성 중 ... 
여기1
여기2
여기1
여기2
여기1
여기2
여기1
여기2
여기1
여기2
🌐 이미지 수집 중 ... 
자기주장의 중요성💬 이미지수집
The Importance of Assertiveness✅ 이미지수집 (1)
자기주장과 타인의 의견💬 이미지수집
🌐 이미지 수집 중 ... 
EMBT 설계방법💬 이미지수집
Assertiveness and the opinions of others✅ 이미지수집 (1)
자기주장을 표현하는 방법💬 이미지수집
How to design an EMBT✅ 이미지수집 (1)
EMBT 프로세스 분석💬 이미지수집
How to express your assertiveness✅ 이미지수집 (1)
🌐 글 형식화 중 ... 
🌐 글 저장 중 ... 
contents saved as JSON :  c:\hellodear\tistory_ebook\bloggers\sweetkiwi\storage\[[sweetkiwi]]_((ko))_``자기주장의 중요성과 표현 방법  자기주장과 타인의 의견  한국어 seo``.json
Data successfully saved to c:\hellodear\tistory_ebook\bloggers\sweetkiwi\for_upload\[[sweetkiwi]]_((ko))_``자기주장의 중요성과 표현 방법  자기주장과 타인의 의견  한국어 seo``.txt
여기4
🌐 글을 작성할 키워드를 

tester 테스트코드

In [ ]:
# 단일블로거로 테스트

# 함수정의
def create_contents(blogger, num_contents_creation=5):
    print(f"Starting thread for {blogger.blogname}")
    blogger.create_contents(num_contents_creation)

# 함수실행

blogger_tester.keyword_ai.vectorstore_save_texts('파이어족 특징') # 예시 키워드 저장
num_of_created_contents_now = len(blogger_tester.file_manager.get_file_names())

try :
    create_contents(blogger_tester, num_contents_creation=1)
finally :
    blogger_tester.keyword_ai.vectorstore_clear()

# 실행결과 확인
num_of_created_contents_after = len(blogger_tester.file_manager.get_file_names())
if (num_of_created_contents_after - num_of_created_contents_now) > 0 :
    print('🌐 컨텐츠 생성이 완료되었습니다.')  


# 클릭3 : 티스토리 블로그 업로드

순차반복문

In [ ]:
# 환경설정
num_contents_upload = int(input('업로드 할 컨텐츠 갯수를 입력하세요. 무한대생산은 엔터를 치세요. '))
assert num_contents_upload is not None, '업로드할 컨텐츠 갯수를 입력하세요.'
assert num_contents_upload <= 15 , '15개 이하로 업로드 가능합니다.'

# 함수정의
def upload(blogger, num_contents_upload):
    blogger.upload_contents(num_contents_upload)

for blogname in blog_names :
    upload(bloggers[blogname], num_contents_upload)


쓰레딩

In [ ]:
# 환경설정
num_contents_upload = int(input('업로드 할 컨텐츠 갯수를 입력하세요. 무한대생산은 엔터를 치세요. '))
assert num_contents_upload is not None, '업로드할 컨텐츠 갯수를 입력하세요.'
assert num_contents_upload <= 15 , '15개 이하로 업로드 가능합니다.'

# 함수정의
def upload(blogger, num_contents_upload):
    blogger.upload_contents(num_contents_upload)

# 실행
thread_bloggers(
    func=upload,
    bloggers=bloggers,
    num_contents_upload=num_contents_upload)

테스트코드 (blogger_tester)

In [ ]:
# 함수정의
def upload(blogger, num_contents_upload = 1):
    blogger.upload_contents(num_contents_upload)

# 함수실행

os.environ["tester_NEW_POST_URL"] = str(r"https://statifi.tistory.com/manage/newpost/?type=post&returnURL=%2Fmanage%2Fposts%2F")
os.environ["tester_ID"] = "jsj950611@naver.com"
os.environ["tester_PW"] = "Zhzkzhffk1!"


upload(blogger_tester, num_contents_upload=1)

# 테스트코드는 여기 이하에 작성

In [ ]:
# 남은 글의 갯수
for blogname in blog_names:
    print(len(bloggers[blogname].file_manager.get_file_names()))

In [ ]:
# 남은 키워드의 갯수
for blogname in blog_names:
    print(len(bloggers[blogname].keyword_ai.vectorstore_list))

In [ ]:
# 이름순서
for blogname in blog_names:
    print(blogname)

In [ ]:
df = bloggers['takuz'].file_manager.get_keywords('collected_keywords')
isNaverIn = []
for all_links in df['all_links'] :
    done = False
    for link in all_links :        
        if link is None : 
            isNaverIn.append(False)
            done = True
            break
        if 'naver' in link :
            isNaverIn.append(True)
            done = True
            break
    if done == False :
        isNaverIn.append(False)
    
        

In [ ]:
df['isNaverIn'] = isNaverIn
len(df[df['isNaverIn'] == True][df['tistory_rank_at_google'] == '노출없음'])

In [ ]:
from module.Crawler import Crawler
crawler = Crawler(isHeadless=False)
crawler

In [ ]:
crawler.search('파이어족', 'google')

In [ ]:
from bs4 import BeautifulSoup
import requests

# 웹페이지 URL
url = crawler.createURL('파이어족', 'daum')
url
response = requests.get(url)
data = response.text

# BeautifulSoup 객체 생성
soup = BeautifulSoup(data, 'html.parser')
soup
# id가 'search'인 요소 찾기
items = soup.find_all('div', class_='c-item-doc')
items

# 각 요소 내의 모든 링크 추출
for item in items:
    links = [a['href'] for a in item.find_all('a', href=True)]
    print(links)

# # 해당 요소 내 모든 링크 추출
# links = [a['href'] for a in search_body.find_all('a', href=True)]

# print(links)



In [ ]:
crawler.search('파이어족', 'daum')

In [ ]:
def
selector = "c-doc-web > div > div.item-title > c-title > strong > a"
crawler.selenium_scroll_action()
crawler.selenium_crawling(selector, get_attribute='href')

In [ ]:
def ddgsearch_reducing(self, keyword, how_many_retrieve=5):
        """웹브라우저 덕덕고로부터 키워드와 관련된 링크를 수집합니다.
        그 다음 해당 링크들로부터 본문을 수집합니다. 수집한 자료는 이후에 RAG에 활용합니다."""
        urls, contents = [], [] # 사이트 주소와 내용을 담을 컨테이너를 준비합니다.
        try:
            selector = "c-doc-web > div > div.item-title > c-title > strong > a"
            self.search(keyword, 'daum')
            urls = crawler.selenium_crawling(selector, get_attribute='href')
            # results = DDGS().text(keyword, max_results=20) # 덕덕고에서 키워드와 관련된 링크를 수집합니다.
            # urls = [a['href'] for a in results if 'html' not in a['href'] and a['href'] != '']
        except Exception as e:
            print(f'{e} with {keyword}')

        
        try:
            idx, clear = 0, 0 # 텍스트 수집에 성공한 갯수를 담을 컨테이너 입니다.
            while clear < how_many_retrieve and idx < len(urls):
                url = urls[idx]
                content = self.get_text_from_webpage(url, self.driver) # 해당 링크로부터 본문을 수집합니다.
                if content:
                    contents.append(self.clean_blog_content(content)) # 수집한 본문을 정제합니다.
                    clear += 1
                idx += 1
        except Exception as e:
            print(e)
        return urls[:how_many_retrieve], contents

In [ ]:
from bs4 import BeautifulSoup
import requests

# 웹페이지 URL
url = crawler.createURL('파이어족', 'daum')
url
response = requests.get(url)
data = response.text

# BeautifulSoup 객체 생성
soup = BeautifulSoup(data, 'html.parser')
soup
# id가 'search'인 요소 찾기
items = soup.find_all('div', class_='c-item-doc')
items

# 각 요소 내의 모든 링크 추출
for item in items:
    links = [a['href'] for a in item.find_all('a', href=True)]
    print(links)

# # 해당 요소 내 모든 링크 추출
# links = [a['href'] for a in search_body.find_all('a', href=True)]

# print(links)



In [ ]:


# for blogname in ['statifi', 'sweetkiwi'] :
#     blogger = bloggers[blogname]
#     blogger.crawler.results = blogger.file_manager.get_keywords('collected_keywords',to_pandas_DataFrame=False)
#     screened_keywords_info = blogger.crawler.load_processed_results()
#     blogger.file_manager.save_keywords('screened_keywords', screened_keywords_info.T.to_dict())
#     blogger.file_manager.save_keywords('suitable_keywords', screened_keywords_info.T.to_dict())
#     print(len(blogger.file_manager.get_keywords('collected_keywords')))
#     print(len(blogger.file_manager.get_keywords('screened_keywords')))
#     print(len(blogger.file_manager.get_keywords('suitable_keywords')))


In [ ]:
df = bloggers['statifi'].file_manager.get_keywords('collected_keywords',to_pandas_DataFrame=False)
bloggers['statifi'].crawler.results = df
bloggers['statifi'].crawler.load_results()

In [ ]:
w = ['리트릿', '요양보호사 건강진단서 유효 기간']
import pandas as pd
pd.DataFrame(df).T.loc[w,]